In [ ]:
# To mount
from google.colab import drive
drive.mount('/content/drive')

# # To initialize the work environment
# %cd /content/drive/My Drive/
# !git clone https://github.com/allnightlight/ConditionalWassersteinAutoencoderPoweredBySinkhornDistance/casestudies -b casestudies

# # To update the work environment
# %cd /content/drive/My Drive/ConditionalWassersteinAutoencoderPoweredBySinkhornDistance
# !git pull

%cd /content/drive/My Drive/ConditionalWassersteinAutoencoderPoweredBySinkhornDistance/casestudies

In [ ]:
import os
import sys
sys.path.append("../framework/")
sys.path.append("../sl/")
sys.path.append("../wae/")

import torch
import numpy as np
import matplotlib 
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D  
import pandas as pd
import itertools

from conc_environment_factory import ConcEnvironmentFactory
from conc_build_parameter import ConcBuildParameter
from conc_build_parameter_factory import ConcBuildParameterFactory
from conc_agent_factory import ConcAgentFactory
from wae_trainer_factory import WaeTrainerFactory

from wae_batch_data_agent import WaeBatchDataAgent
from wae_batch_data_environment import WaeBatchDataEnvironment

from builder import Builder
from store import Store
from mylogger import MyLogger

from loader import Loader

In [ ]:
dbPath = "training_log.sqlite"

In [ ]:
target_casestudy = "cs02a"

In [ ]:
def evaluate_err(buildParameter, agent):
    environment = environmentFactory.create(buildParameter)
    trainer = trainerFactory.create(buildParameter, agent, environment)

    dataBatchEnv = environment.getTestData()
    dataBatchAg = agent(dataBatchEnv)

    _err_observable = torch.mean(torch.abs(dataBatchAg._XHat - dataBatchEnv._X))
    _err_latent, _ = trainer.measure_distance(dataBatchAg._Xi, dataBatchAg._XiHat)

    err_observable = _err_observable.data.numpy()
    err_latent = _err_latent.data.numpy()    
    return err_observable, err_latent

In [ ]:
def plot_encoder_projection_image(agent, environment, azim):

    def f(r, t):
        x = (r * np.cos(t) + 2) * np.cos(2*t)
        y = (r * np.cos(t) + 2) * np.sin(2*t)
        z = r * np.sin(t)
        X = np.stack((x,y,z), axis=-1) # (*, 3)
        return X
    # r = self.randomState.rand(nSample) * 2 - 1
    # t = self.randomState.rand(nSample) * np.pi 

    nR = 2**6
    nT = 2**6
    r = np.linspace(-1, 1, nR)
    t = np.linspace(0, 1, nT) * np.pi
    R, T = np.meshgrid(r, t) # (nT, nR)
    XYZ = f(R, T) # (nT, nR, 3)

    _XYZ = torch.from_numpy(XYZ.astype(np.float32).reshape(-1,3)) # (nT * nR, 3)

    _Z = torch.ones(nT * nR).reshape(-1,1) # (*, nZ = 1)

    dataBatchEnv = WaeBatchDataEnvironment(_XYZ, _Z)
    dataBatchAgent = agent(dataBatchEnv)
    XiHat = dataBatchAgent._XiHat.data.numpy().reshape(nT, nR, -1) # (nTheta, nR, 3)

    #
    nZ = 2**6
    nPh = 2**6
    z = np.linspace(0,1,nZ)
    ph = np.linspace(0,1,nPh)*2*np.pi
    Z, Ph = np.meshgrid(z, ph) # (nPh, nZ)
    Xi = np.stack((np.cos(Ph), np.sin(Ph), Z), axis=-1) # (nPh, nZ, 3)

    #
    V = R * np.sin(T)
    norm = matplotlib.colors.Normalize(vmin=V.min().min(), vmax=V.max().max())
    cmp = plt.get_cmap("Blues")
    colors = cmp(norm(V))

    fig = plt.gcf()
    #
    ax = fig.add_subplot(1,2,1, projection = "3d")
    ax.plot_surface(XYZ[...,0], XYZ[...,1], XYZ[...,2], facecolors=colors, linewidth = 0, antialiased=True, shade = False)
    ax.set_title('Input image \non the observable variables space')
    ax.axis('off')
    ax.view_init(60, 300)
    fig.tight_layout()
    #
    ax = fig.add_subplot(1,2,2, projection = "3d")
    ax.plot_wireframe(Xi[...,0], Xi[...,1], Xi[...,2], color='lightgray', linewidth=1.0, alpha = .5)
    ax.plot_surface(XiHat[...,0], XiHat[...,1], XiHat[...,2], facecolors=colors, linewidth = 0, antialiased=True, shade = False)
    ax.set_title('Ouput image \non the latent variables space')
    ax.axis('off')
    ax.view_init(60, azim)
    #
    ax.set_xlim(-1.10,1.10)
    ax.set_ylim(-1.10,1.10)
    ax.set_zlim(-0.10,1.10)
    fig.tight_layout()

In [ ]:
def plot_decoder_projection_image(agent, environment, azim):

    def f(r, t):
        x = (r * np.cos(t) + 2) * np.cos(2*t)
        y = (r * np.cos(t) + 2) * np.sin(2*t)
        z = r * np.sin(t)
        X = np.stack((x,y,z), axis=-1) # (*, 3)
        return X
    # r = self.randomState.rand(nSample) * 2 - 1
    # t = self.randomState.rand(nSample) * np.pi 

    nR = 2**6
    nT = 2**6
    r = np.linspace(-1, 1, nR)
    t = np.linspace(0, 1, nT) * np.pi
    R, T = np.meshgrid(r, t) # (nT, nR)
    XYZ = f(R, T) # (nT, nR, 3)

    #
    nZ = 2**6
    nPh = 2**6
    z = np.linspace(0,1,nZ)
    ph = np.linspace(0,1,nPh)*2*np.pi
    Z, Ph = np.meshgrid(z, ph) # (nPh, nZ)
    Xi = np.stack((np.cos(Ph), np.sin(Ph), Z), axis=-1) # (nPh, nZ, 3)

    _Xi = torch.from_numpy(Xi.astype(np.float32).reshape(-1, 3)) # (nPh * nZ, 3)
    _XYZhat = agent.dec(_Xi) # (nPh * nZ, 3)
    XYZhat = _XYZhat.data.numpy().reshape(nPh, nZ, 3) # (nPh, nZ, 3)

    #
    V = Z
    norm = matplotlib.colors.Normalize(vmin=V.min().min(), vmax=V.max().max())
    cmp = plt.get_cmap("Reds")
    colors = cmp(norm(V))

    fig = plt.gcf()
    #
    ax = fig.add_subplot(1,2,1, projection = "3d")
    ax.plot_surface(Xi[...,0], Xi[...,1], Xi[...,2], facecolors=colors, linewidth = 0, antialiased=True, shade = False)

    ax.set_title('Input image \non the latent variables space')
    ax.axis('off')
    ax.view_init(60, 300)
    ax.set_xlim(-1.10,1.10)
    ax.set_ylim(-1.10,1.10)
    ax.set_zlim(-0.10,1.10)
    fig.tight_layout()
    #
    ax = fig.add_subplot(1,2,2, projection = "3d")
    ax.plot_wireframe(XYZ[...,0], XYZ[...,1], XYZ[...,2], color='lightgray', linewidth=1.0, alpha = .5)
    ax.plot_surface(XYZhat[...,0], XYZhat[...,1], XYZhat[...,2], facecolors=colors, linewidth = 0, antialiased=True, shade = False)

    ax.set_title('Ouput image \non the observable variables space')
    ax.axis('off')
    ax.view_init(60, azim)
    #
    fig.tight_layout()

## S400: Load trained agents to analyze them

### SS410: initialize a loader of trained agents

In [ ]:
agentFactory = ConcAgentFactory()
environmentFactory = ConcEnvironmentFactory()
trainerFactory =  WaeTrainerFactory()

store = Store(dbPath)

buildParameterFactory = ConcBuildParameterFactory()
loader = Loader(agentFactory=agentFactory
                , environmentFactory=environmentFactory
                , buildParameterFactory=buildParameterFactory
                , store = store)

### SS420: evaluate trained agents:

evaluation error is here:
* the representitive errors of observable variables
* and the discrepancy between latent referenced distribution and the one projected by trained encoder

In [ ]:
tbl = {
    "criteria": []
    , "score": []
    , "epoch": []
      }
for agent, buildParameter, epoch in loader.load(target_casestudy + "%"):
    representative_error, latent_distribution_discrepancy = evaluate_err(buildParameter, agent)
    
    for score, criteria in [
        (representative_error, "Representative Error")
        , (latent_distribution_discrepancy, "Latent Distributions Discrepancy")]:

        for key in buildParameter.__dict__:
            if not key in tbl:
                tbl[key] = []
            tbl[key].append(buildParameter.__dict__[key])
    
        tbl["epoch"].append(epoch)
        tbl["criteria"].append(criteria)
        tbl["score"].append(score)

tbl = pd.DataFrame(tbl)
tbl.to_csv(target_casestudy +  "_score.csv")

### SS430: Print encoder's images

In [ ]:
for agent, buildParameter, epoch in loader.load(target_casestudy + "%", buildParameterKey=None):
    
    if epoch == buildParameter.nEpoch:
        for azim in np.arange(0, 360, 30):
            environment = environmentFactory.create(buildParameter)
            fig = plt.figure(figsize=[12, 6])
            plot_encoder_projection_image(agent, environment, azim)
            fig.savefig("./img/cs02a_encoder_projection_%s_azim=%03d.png" % (buildParameter.key, azim))
            plt.close(fig)
        break

### SS440: Print decoder's images

In [ ]:
for agent, buildParameter, epoch in loader.load(target_casestudy + "%", buildParameterKey=None):
    
    if epoch == buildParameter.nEpoch:
        for azim in np.arange(0, 360, 30):        
            environment = environmentFactory.create(buildParameter)
            fig = plt.figure(figsize=[12, 6])
            plot_decoder_projection_image(agent, environment, azim)
            fig.savefig("./img/cs02a_deccoder_projection_%s_azim=%03d.png" % (buildParameter.key, azim))
            plt.close(fig)
        break